# Sprawdzenie i oczyszczenie danych

In [1]:
#Importuję potrzebne biblioteki. 
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#Importuję zbiór danych.
df1 = pd.read_excel("Dataset Basic.xlsx")

In [2]:
#Sprawdzam jak wygląda zbiór danych.
#df1.head(1000)

In [3]:
#Sprawdzam podstawowe statystyki dotyczące zmiennych.
df1.describe()

,Z9,Z10,Z11,Z12,Z13,Z14,Z15,Z16,Z17,Z18,Z19,Z20,Z21
count,640,639,638,639,382,551,273,58,36,63,7,9,502
unique,4,4,31,4,375,105,45,12,17,14,5,4,88
top,1,0,1,0,0,1,1,1,1,1,1,1,2
freq,443,620,265,521,6,72,75,36,10,38,3,6,43


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Z9      640 non-null    object
 1   Z10     639 non-null    object
 2   Z11     638 non-null    object
 3   Z12     639 non-null    object
 4   Z13     382 non-null    object
 5   Z14     551 non-null    object
 6   Z15     273 non-null    object
 7   Z16     58 non-null     object
 8   Z17     36 non-null     object
 9   Z18     63 non-null     object
 10  Z19     7 non-null      object
 11  Z20     9 non-null      object
 12  Z21     502 non-null    object
dtypes: object(13)
memory usage: 65.5+ KB


In [5]:
df1 = df1.drop('Z13',axis=1) #Usuwam zmienną Z13, ponieważ nie będzie wykorzystywana w sieci. 
df1 = df1.drop([0,1]) #Usuwam pierwsze dwa wiersze, które służą jako opis dla zmiennych.

df1['Z9'] = df1['Z9'].astype(bool) #Zmieniam typ zmiennych, które przyjmują wartości 0 lub 1 na boolean (wartości logiczne - prawda lub fałsz).
df1['Z10'] = df1['Z9'].astype(bool)
df1['Z12'] = df1['Z9'].astype(bool)

#Po przeglądnięciu zbioru postanowiłem, że najlepszym sposobem na pozbycie się brakujących danych będzie zamienienie ich na 0.
#Przyjmuję więc, że brak danych jest równoznaczny z tym, że reakcji/obrazu po prostu nie było.
df1['Z11'].fillna(0, inplace=True) 
df1['Z14'].fillna(0, inplace=True)
df1['Z15'].fillna(0, inplace=True)
df1['Z16'].fillna(0, inplace=True)
df1['Z17'].fillna(0, inplace=True)
df1['Z18'].fillna(0, inplace=True)
df1['Z19'].fillna(0, inplace=True)
df1['Z20'].fillna(0, inplace=True)
df1['Z21'].fillna(0, inplace=True)

#Resetuję index w związku z usunięciem dwóch pierwszych wierszy.
df1 = df1.reset_index() 
df1 = df1.drop('index',axis=1)

In [6]:
#Tak wygląda gotowy zbiór danych.
#df1.head(1000)

In [7]:
df1.describe()

,Z11,Z14,Z15,Z16,Z17,Z18,Z19,Z20,Z21
count,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000,642.000000
mean,2.376947,26.512461,4.766355,0.903427,0.506231,0.409657,0.461059,0.009346,17.526480
std,5.840947,95.635834,24.150818,18.329963,4.438253,4.178778,11.563804,0.096296,41.950235
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
75%,2.000000,18.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000
max,78.000000,1600.000000,472.000000,464.000000,81.000000,99.000000,293.000000,1.000000,547.000000


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Z9      642 non-null    bool 
 1   Z10     642 non-null    bool 
 2   Z11     642 non-null    int64
 3   Z12     642 non-null    bool 
 4   Z14     642 non-null    int64
 5   Z15     642 non-null    int64
 6   Z16     642 non-null    int64
 7   Z17     642 non-null    int64
 8   Z18     642 non-null    int64
 9   Z19     642 non-null    int64
 10  Z20     642 non-null    int64
 11  Z21     642 non-null    int64
dtypes: bool(3), int64(9)
memory usage: 47.1 KB


In [9]:
#Dodatkowo sprawdzam w Excelu czy wszystko się zgadza.
df1.to_excel('test.xlsx')

# Pierwsza Iteracja

### Wczytywanie danych do modelu

In [10]:
#Konwertuję zmienne na tablicę numpy.
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values #Zmienne na podstawie których przewidujemy (cechy wejściowe). 
y = df1['Z21'].values #Liczba komentarzy, którą chcemy przewidywać (wektor docelowy).

### Podział danych na zbiór treningowy i testowy

In [11]:
#Zbiór treningowy - 80%, Zbiór testowy - 20%, 'random_state=42' zapewnia powtarzalność podziału danych.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Skalowanie zmiennych

In [12]:
#Standaryzuję zbiór treningowy za pomocą StandardScaler z biblioteki sklearn. 
#Następnie przeprowadzam takie samo skalowanie na zbiorze testowym, zapewniając spójność między obydwoma zbiorami.
#Standardyzacja przekształca dane tak, aby miały średnią równą zero i wariancję równą jeden.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Przygotowanie architektury sieci (Perceptron)
Definiuje klasę 'Perceptron', która implementuje algorytm perceptronu złożony z pojedynczego sztucznego neuronu. Jego celem jest przewidywanie liczby komentarzy na podstawie podanych zmiennych. 

In [13]:
class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100): 
        self.num_features = num_features #Liczba cech wejściowych.
        self.learning_rate = learning_rate #Współczynnik uczenia (jak szybko model będzie się dostosowywał do danych treningowych).
        self.num_epochs = num_epochs #Licza epok (ile razy model przejdzie przez cały zbiór danych treningowych podczas procesu uczenia).
        self.weights = np.zeros(num_features + 1) #Wagi początkowe - wektor zer o długości = liczba cech + 1 (dodatkowa jedynka odnosi się do wagi biasu)
        print("Wagi początkowe:")
        print(self.weights) #Wypisuje wagi początkowe modelu.
        
    def get_weights(self):
        return self.weights #Zwraca aktualne wagi modelu.
    
    def activate(self, x):
            return x #Funkcja aktywacji, która zwraca input (x) bez żadnych zmian.

    def predict(self, x): #Służy do przewidywania wyniku na podstawie podanego wektora wejściowego x.
        x = np.insert(x, 0, 1)  #Dodanie 1 na pozycji zerowej odpowiada za bias.
        activation = np.dot(self.weights, x) #Funkcja np.dot z biblioteki NumPy wykonuje iloczyn skalarany między wektorem wag, a wektorem x.
        return activation #Wynik zwraca wartość liczbową (aktywację), która stanowi naszą predykcję.
    
    def train(self, X, y): #Trenowanie perceptronu poprzez aktualizację wag na podstawie błędu predykcji.
        for _ in range(self.num_epochs): #Pętla iterująca przez określoną liczbę epok.
            for i in range(len(X)): #Pętla iterująca przez wszystkie przykłady treningowe ('i' to indeks bieżącego przykładu).
                x = X[i] #Wektor wejściowy (x) dla danego przykładu.
                y_true = y[i] #Wartość docelowa dla danego przykładu.
                y_pred = self.predict(x) #Obliczenie predykcji dla danego x. 
                error = y_true - y_pred #Obliczenie błędu predykcji jako różnicy między wartością docelową 'y_true' a predykcją 'y_pred'.
                self.weights += self.learning_rate * error * np.insert(x, 0, 1) #Aktualizacja wag perceptronu na podstawie błędu predykcji.

### Inicjalizacja i trening perceptronu

In [14]:
perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100) #Ustawia liczbę cech na równą liczbie kolumn w zbiorze treningowym + określa współczynnik uczenia i liczbę epok. 
perceptron.train(X_train_scaled, y_train) #Uczenie modelu za pomocą metody 'train' poprzez przekazanie standaryzowanych danych treningowych oraz danych docelowych.

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Prognozowanie na danych treningowych i testowych

In [15]:
#Generuje przewidywane wartości na podstawie wytrenowanego perceptronu dla danych treningowych i testowych.
y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]
#print(y_pred_train) #lista przewidywanych wartości dla danych treningowych.
#print(y_pred_test) #lista przewidywanych wartości dla danych testowych.

### Obliczanie błędu średniokwadratowego (MSE)

In [16]:
#Miara oceny jakości modelu
#Oblicza różnicę pomiędzy przewidywanymi wartościami a rzeczywistymi wartościami osobno w zbiorze treningowym i zbiorze testowym.
#Różnice zostają podniesione do kwadratu i obliczana jest ich średnia wartość.
mse_train = np.mean((y_pred_train - y_train) ** 2) 
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (treningowe):', mse_train)
print('Błąd (testowe):', mse_test)

Błąd (treningowe): 3.4578085077500045e+212
Błąd (testowe): 7.549259127633164e+210


Wartości błędu są bardzo wysokie, co sprawia, że obecna architektura nie będzie przydatna w tworzeniu predykcji. 

### Wagi

In [17]:
trained_weights = perceptron.get_weights() #pobiera wagi modelu, które są rezultatem procesu uczenia (wraz z wagą biasu).
print("Wagi końcowe:")
print(trained_weights)

Wagi końcowe:
[-4.21821224e+104  3.63378053e+103  3.63378053e+103 -9.49304435e+103
  3.63378053e+103 -4.41918151e+105  7.85555541e+105 -7.22310964e+105
  2.03371002e+103 -1.27995074e+105 -7.66462458e+105 -2.34631615e+104]


Wartości wag reprezenują wpływ (negatywny lub pozytywny) danej zmiennej na liczbę komentarzy oraz siłę tego wpływu.

### Przykład działania na nowych danych

In [18]:
#Nowe dane
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])

#Standaryzacja nowych danych
new_data_scaled = scaler.transform(new_data)

#Przewidywanie liczby komentarzy
prediction = perceptron.predict(new_data_scaled)

#Wyświetlenie przewidywanej liczby komentarzy
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.997557902257973e+105


Przewidywana liczba komentarzy jest w oczywisty sposób nietrafiona.

# Kolejne Iteracje - wykaz sprawdzonych architektur sieci

## Badanie wpływu zmiany typu inputów na wyniki

### Zmiana na Integer (liczby całkowite)

In [19]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights
    
    def activate(self, x):
        return x
    
    #Dodanie funkcji, która konwertuje każdy input x na liczby całkowite (integer).
    def preprocess_input(self, x):
        processed_input = [int(feature) for feature in x]
        return processed_input
    
    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation  

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 3.4578085077500045e+212
Błąd (Testowe): 7.549259127633164e+210
Wagi końcowe:
[-4.21821224e+104  3.63378053e+103  3.63378053e+103 -9.49304435e+103
  3.63378053e+103 -4.41918151e+105  7.85555541e+105 -7.22310964e+105
  2.03371002e+103 -1.27995074e+105 -7.66462458e+105 -2.34631615e+104]


Ani wartości błędu ani wagi końcowe nie zmieniły się - wyniki nadal nie są przydatne.

In [20]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.997557902257973e+105


### Zmiana na Float (liczby zmiennoprzecinkowe)

In [21]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x
    
    #Dodanie funkcji, która konwertuje każdy input x na liczby zmiennoprzecinkowe (float).
    def preprocess_input(self, x):
        processed_input = [float(feature) for feature in x]
        return processed_input

    def predict(self, x): 
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 3.4578085077500045e+212
Błąd (Testowe): 7.549259127633164e+210
Wagi końcowe:
[-4.21821224e+104  3.63378053e+103  3.63378053e+103 -9.49304435e+103
  3.63378053e+103 -4.41918151e+105  7.85555541e+105 -7.22310964e+105
  2.03371002e+103 -1.27995074e+105 -7.66462458e+105 -2.34631615e+104]


Ani wartości błędu ani wagi końcowe nie zmieniły się - wyniki nadal nie są przydatne.

In [22]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.997557902257973e+105


### Zmiana na String (wartość tekstowa)

In [23]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x
    
    #Dodanie funkcji, która konwertuje każdy input x na wartość tekstową (string).
    def preprocess_input(self, x):
        processed_input = [string(feature) for feature in x]
        return processed_input

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 3.4578085077500045e+212
Błąd (Testowe): 7.549259127633164e+210
Wagi końcowe:
[-4.21821224e+104  3.63378053e+103  3.63378053e+103 -9.49304435e+103
  3.63378053e+103 -4.41918151e+105  7.85555541e+105 -7.22310964e+105
  2.03371002e+103 -1.27995074e+105 -7.66462458e+105 -2.34631615e+104]


Ani wartości błędu ani wagi końcowe nie zmieniły się - wyniki nadal nie są przydatne.

In [24]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.997557902257973e+105


## Badanie wpływu zmiany ilości inputów na wyniki

### Inputy związane z obrazkami i grafiką 

In [25]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12']].values #Wybiera tylko pierwsze cztery zmienne jako input.
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0.]
Błąd (Treningowe): 1248.9997506177065
Błąd (Testowe): 3667.0648727800813
Wagi końcowe:
[15.32214127 -0.87995224 -0.87995224 -2.07605293 -0.87995224]


Wartość błędu jest znacznie niższa niż w poprzednich przypadkach, co sprawia, że można użyć zmiennych dotyczących grafiki i obrazków w przewidywaniu liczby komentarzy.

Wagi końcowe wskazują, że zmienna logiczna dotycząca obecności (lub braku) grafiki ma decydujący (pozytywny) wpływ na liczbę komentarzy. Nieznacznie negatywny wpływ ma natomiast fakt, że obrazki są tylko online. Reszta zmiennych nie ma większego znaczenia. 

In [26]:
new_data = np.array([[False, True, 3, False]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 17.099158982879082


Przewidywana liczba komentarzy wydaje się być znacznie bardziej prawdopodobna.

### Inputy związane z reakcjami

In [27]:
X = df1[['Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values #Wybiera tylko zmienne związane z reakcjami jako input.
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x
    
    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 3.9332821978214815e+212
Błąd (Testowe): 8.570302834389428e+210
Wagi końcowe:
[-4.54643811e+104 -4.67308899e+105  8.35130042e+105 -7.72102106e+105
  1.90976603e+103 -1.29716793e+105 -8.18843644e+105 -2.27586065e+104]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

In [28]:
new_data = np.array([[32, 52, 12, 14, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 7.677825596125923e+105


## Badanie wpływu zmiany sposobu generowania wag początkowych na wyniki

### Losowe wartości z rozkładu jednorodnego

In [29]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        #Nowe wagi początkowe.
        #Generuje losowe liczby z rozkładu jednorodnego między -1 a 1, zwracając tablicę o rozmiarze = liczba cech + 1.
        self.weights = np.random.uniform(-1, 1, size=num_features + 1) 
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[ 0.46316904 -0.25632238 -0.88559482  0.25019777 -0.30287936  0.07899875
  0.87524405  0.61219516  0.55185888  0.14663932 -0.11837609 -0.10028042]
Błąd (Treningowe): 3.391551967058672e+212
Błąd (Testowe): 7.404604560019464e+210
Wagi końcowe:
[-4.17760329e+104  3.59879793e+103  3.59879793e+103 -9.40165431e+103
  3.59879793e+103 -4.37663781e+105  7.77992957e+105 -7.15357238e+105
  2.01413138e+103 -1.26762859e+105 -7.59083684e+105 -2.32372804e+104]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

In [30]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.90131088338964e+105


### Losowe wartości z rozkładu normalnego

In [31]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        #Nowe wagi początkowe.
        #Generuje losowe liczby z rozkładu normalnego o średniej równiej 0 i odchyleniu standardowym równym 1, zwracając tablicę o rozmiarze = liczba cech + 1.
        self.weights = np.random.normal(0, 1, size=num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)


Wagi początkowe:
[-1.40778236 -0.18788927 -1.37830444  0.14021934  1.1306861  -0.86541444
 -0.8324827   0.7472862  -0.16699829  0.63724627  0.67287172 -0.34900768]
Błąd (Treningowe): 3.2264692325961906e+212
Błąd (Testowe): 7.044187741921459e+210
Wagi końcowe:
[-4.07466325e+104  3.51012019e+103  3.51012019e+103 -9.16998879e+103
  3.51012019e+103 -4.26879338e+105  7.58822485e+105 -6.97730169e+105
  1.96450131e+103 -1.23639304e+105 -7.40379154e+105 -2.26646921e+104]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

In [32]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.657333356644634e+105


### Metoda Xavier/Glorot 

In [33]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        #Nowe wagi początkowe.
        limit = np.sqrt(6 / (num_features + 1)) #Limit wartości wag na podstawie liczby cech wejściowych.
        self.weights = np.random.uniform(-limit, limit, size=num_features + 1) #Generuje losowe wagi z rozkładu jednostajnego w przedziale [-limit, limit].
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[-0.00476648 -0.01719371 -0.28053543  0.4559279  -0.29978026  0.42124367
 -0.24231752 -0.51858146 -0.47336219  0.05976648  0.49625791  0.66880925]
Błąd (Treningowe): 3.3562886267358822e+212
Błąd (Testowe): 7.327615885485903e+210
Wagi końcowe:
[-4.15582842e+104  3.58003996e+103  3.58003996e+103 -9.35265020e+103
  3.58003996e+103 -4.35382552e+105  7.73937834e+105 -7.11628591e+105
  2.00363315e+103 -1.26102134e+105 -7.55127122e+105 -2.31161610e+104]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

In [34]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 9.84970241330213e+105


## Badanie wpływu zmiany sposobu przetwarzania danych wejściowych na wyniki

### Normalizacja danych wejściowych
Zastosowanie funkcji MinMaxScaler jako skalera zamiast funkcji StandardScaler.

In [35]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Wykonuje normalizację danych przy użyciu skalera typu MinMaxScaler z biblioteki scikit-learn.
#Przekształca wartości danych w taki sposób, aby mieściły się w zakresie (0, 1).
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_normalized.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_normalized, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_normalized]
y_pred_test = [perceptron.predict(x) for x in X_test_normalized]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 649.2942800148435
Błąd (Testowe): 3123.2055766658464
Wagi końcowe:
[ 22.64471859  -4.89679371  -4.89679371 -20.470875    -4.89679371
 179.63017432  20.21922841 159.02609152  14.71578482  36.36817021
 157.32006213  -0.47633756]


Wartość błędu jest najniższa ze wszystkich dotychczasowych przypadków.

Wagi końcowe pokazują następujące zależności:
- Największy pozytywny wpływ na liczbę komentarzy mają zmienne dotyczące reakcji "LOVE", "HAHA" i "WRR".
- Mniejszy pozytywny wpływ mają zmienne dotyczące obecności grafiki oraz reakcji "CRY", "WOW" i "HUG".
- Największy negatywny wpływ na liczbę komentarzy ma zmienna, która wskazuję na to czy obrazki są dostępne tylko online.
- Wpływ reszty zmiennych jest stosunkowo marginalny.

In [36]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 53.78529467215922


Na tle dotychczasowych iteracji, wynik proponowany przez tę architekturę wydaję się być najbardziej realistyczny. 

### Przekształcenie logarytmiczne danych wejściowych

In [37]:
df1['Z9'] = df1['Z9'].astype('int64')
df1['Z10'] = df1['Z9'].astype('int64')
df1['Z12'] = df1['Z9'].astype('int64')

X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Logarytmowanie danych treningowych i testowych za pomocą funkcji log1p z biblioteki NumPy.
X_train_log = np.log1p(X_train)
X_test_log = np.log1p(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_log.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_log, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_log]
y_pred_test = [perceptron.predict(x) for x in X_test_log]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 622.5841139734817
Błąd (Testowe): 3168.280043841729
Wagi końcowe:
[14.23814908 -8.9054584  -8.9054584  -6.4065441  -8.9054584   8.65545831
 -6.06927855 16.75857496  0.09396205  1.81596855 70.1839925   0.4945371 ]


Przekształcenie logarytmiczne również wydaje się znacznie poprawiać otrzymywane wyniki. W porównaniu do normalizacji ten sposób przetwarzania danych wejściowych nieznacznie zmniejsza błąd w danych treningowych, jednocześnie nieznacznie zwiększając błąd w danych testowych.

Wagi wskazują, że na liczbę komentarzy pozytywne wpływa przede wszystkim ilość reakcji "WRR", a w dalszej kolejność ilość reakcji "HAHA", "LOVE" oraz obecność grafiki. 

In [38]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_log = np.log1p(new_data)
prediction = perceptron.predict(new_data_log)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 105.86112959926703


Przewidywana liczba komentarzy jest dwa razy większa niż w poprzedniej architekturze.

## Badanie wpływu zmiany sposobu przetwarzania danych wyjściowych na wyniki

### Standaryzacja

In [39]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

#Standaryzacja danych wyjściowych
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train_scaled)

y_pred_train = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train- y_train) ** 2)
mse_test = np.mean((y_pred_test- y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 2.3899576010134094e+209
Błąd (Testowe): 5.217874036016803e+207
Wagi końcowe:
[-1.10897832e+103  9.55329793e+101  9.55329793e+101 -2.49574459e+102
  9.55329793e+101 -1.16181363e+104  2.06524474e+104 -1.89897320e+104
  5.34667340e+101 -3.36502182e+103 -2.01504856e+104 -6.16852257e+102]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

### Normalizacja 

In [40]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

#Normalizacja danych wyjściowych
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train_scaled)

y_pred_train_scaled = np.array([perceptron.predict(x) for x in X_train_scaled])
y_pred_test_scaled = np.array([perceptron.predict(x) for x in X_test_scaled])

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 2.3899576010134094e+209
Błąd (Testowe): 5.217874036016803e+207
Wagi końcowe:
[-7.86979896e+101  6.77944128e+100  6.77944128e+100 -1.77109036e+101
  6.77944128e+100 -8.24474162e+102  1.46558870e+103 -1.34759508e+103
  3.79423510e+100 -2.38796780e+102 -1.42996727e+103 -4.37745550e+101]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

### Przekształcenie logarytmiczne

In [41]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

#Logarytmowanie danych wyjściowych
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train_log)

y_pred_train_log = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test_log = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 2.3899576010134094e+209
Błąd (Testowe): 5.217874036016803e+207
Wagi końcowe:
[-1.31078578e+102  1.12917690e+101  1.12917690e+101 -2.94991025e+101
  1.12917690e+101 -1.37323585e+103  2.44106975e+103 -2.24454078e+103
  6.31963973e+100 -3.97737508e+102 -2.38173907e+103 -7.29104573e+101]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

## Badanie wpływu zmiany sposobu przetwarzania danych wejściowych i wyjściowych na wyniki

### Normalizacja danych wejściowych i wyjściowych

In [42]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Normalizacja danych wejściowych
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

#Normalizacja danych wyjściowych
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train_scaled)

y_pred_train_scaled = np.array([perceptron.predict(x) for x in X_train_scaled])
y_pred_test_scaled = np.array([perceptron.predict(x) for x in X_test_scaled])

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 2.3899576010134094e+209
Błąd (Testowe): 5.217874036016803e+207
Wagi końcowe:
[-7.86979896e+101  6.77944128e+100  6.77944128e+100 -1.77109036e+101
  6.77944128e+100 -8.24474162e+102  1.46558870e+103 -1.34759508e+103
  3.79423510e+100 -2.38796780e+102 -1.42996727e+103 -4.37745550e+101]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

### Przekształcenie logarytmiczne wejściowych i wyjściowych

In [43]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Logarytmowanie danych wejściowych
X_train_log = np.log1p(X_train)
X_test_log = np.log1p(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.weights = np.zeros(num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

#Logarytmowanie danych wyjściowych
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

perceptron = Perceptron(num_features=X_train_scaled.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_scaled, y_train_log)

y_pred_train_log = [perceptron.predict(x) for x in X_train_scaled]
y_pred_test_log = [perceptron.predict(x) for x in X_test_scaled]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Błąd (Treningowe): 2.3899576010134094e+209
Błąd (Testowe): 5.217874036016803e+207
Wagi końcowe:
[-1.31078578e+102  1.12917690e+101  1.12917690e+101 -2.94991025e+101
  1.12917690e+101 -1.37323585e+103  2.44106975e+103 -2.24454078e+103
  6.31963973e+100 -3.97737508e+102 -2.38173907e+103 -7.29104573e+101]


Wartości błędu są bardzo wysokie, co sprawia, że ta architektura nie będzie przydatna w tworzeniu predykcji.

# Finalna architektura

In [44]:
X = df1[['Z9', 'Z10', 'Z11', 'Z12', 'Z14', 'Z15', 'Z16', 'Z17', 'Z18', 'Z19', 'Z20']].values
y = df1['Z21'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Normalizacja danych wejściowych przy użyciu skalera typu MinMaxScaler.
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

class Perceptron:
    def __init__(self, num_features, learning_rate=0.01, num_epochs=100):
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        #Losowe wartości z rozkładu normalnego jako wagi początkowe.
        self.weights = np.random.normal(0, 1, size=num_features + 1)
        print("Wagi początkowe:")
        print(self.weights)
        
    def get_weights(self):
        return self.weights

    def activate(self, x):
        return x

    def predict(self, x):
        x = np.insert(x, 0, 1)  
        activation = np.dot(self.weights, x)
        return activation

    def train(self, X, y):
        for _ in range(self.num_epochs):
            for i in range(len(X)):
                x = X[i]
                y_true = y[i]
                y_pred = self.predict(x)
                error = y_true - y_pred
                self.weights += self.learning_rate * error * np.insert(x, 0, 1)

perceptron = Perceptron(num_features=X_train_normalized.shape[1], learning_rate=0.01, num_epochs=100)
perceptron.train(X_train_normalized, y_train)

y_pred_train = [perceptron.predict(x) for x in X_train_normalized]
y_pred_test = [perceptron.predict(x) for x in X_test_normalized]

mse_train = np.mean((y_pred_train - y_train) ** 2)
mse_test = np.mean((y_pred_test - y_test) ** 2)

print('Błąd (Treningowe):', mse_train)
print('Błąd (Testowe):', mse_test)

trained_weights = perceptron.get_weights()
print("Wagi końcowe:")
print(trained_weights)

Wagi początkowe:
[-0.75877391 -0.01043738  1.72326738 -0.17073291 -0.40685738 -0.63302118
 -0.07536434  0.59451535  0.22301761  0.1512672  -1.30690894  0.54501815]
Błąd (Treningowe): 649.2985420388036
Błąd (Testowe): 3123.27578099641
Wagi końcowe:
[ 22.64486099  -5.34182676  -3.608122   -20.47909052  -5.73824676
 179.46125369  20.25523492 160.01880244  14.75047674  36.46792395
 156.41042624  -0.44806161]


Tak opracowana architektura ma najmniejszy błąd średniokwadratowy. <br>
- Wykorzystuje normalizacje danych wejściowych za pomocą MinMaxScaler
- Wagi początkowe to losowe wartości z rozkładu normalnego 

In [45]:
new_data = np.array([[False, True, 3, False, 11, 50, 5, 23, 1, 2, 1]])
new_data_scaled = scaler.transform(new_data)
prediction = perceptron.predict(new_data_scaled)
print('Przewidywana liczba komentarzy:', prediction)

Przewidywana liczba komentarzy: 55.182755104148505


## Zestaw finalnie opracowanych wag

Wagi końcowe:
[ 22.64486099  -5.34182676  -3.608122   -20.47909052  -5.73824676 <br>
 179.46125369  20.25523492 160.01880244  14.75047674  36.46792395 <br>
 156.41042624  -0.44806161]
 
 Wagi końcowe pokazują następujące zależności:
- Największy pozytywny wpływ na liczbę komentarzy mają zmienne dotyczące reakcji "LOVE", "HAHA" i "WRR".
- Mniejszy pozytywny wpływ mają zmienne dotyczące obecności grafiki oraz reakcji "CRY", "WOW" i "HUG".
- Największy negatywny wpływ na liczbę komentarzy ma zmienna, która wskazuję na to czy obrazki są dostępne tylko online.
- Wpływ reszty zmiennych jest stosunkowo marginalny.